### stage_model

prediction models on stage dataset

data format:
    target , eventid ,    car_number,    stageid,     features...

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# to use only one GPU.
# use this on r-001
# otherwise comment
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
#import xgboost as xgb


In [38]:
# bulid regression model
def get_classifier(classifier = 'lr'):
    
    class_weight = None
    
    if classifier == "lsvc":
        clf = LinearSVC(penalty='l1',dual=False, tol=1e-3, class_weight=class_weight )
    elif classifier == "lsvcl2":
        clf = LinearSVC(penalty='l2', tol=1e-4, class_weight=class_weight)
    elif classifier == 'rf':
        #clf = RandomForestClassifier(n_estimators=100, n_jobs=4,criterion='entropy', min_samples_split=1,class_weight = class_weight)
        clf = RandomForestClassifier(n_estimators=100, n_jobs=-1,criterion='entropy', class_weight = class_weight)
    elif classifier == 'lr':
        clf = LogisticRegression(class_weight = class_weight, n_jobs=-1, fit_intercept = False, verbose = 0)
    elif classifier == 'lrbias':
        clf = LogisticRegression(class_weight = class_weight, n_jobs=-1, fit_intercept = True, verbose = 1)
    elif classifier == 'lrl1':
        clf = LogisticRegression(class_weight = class_weight, penalty='l1',n_jobs=-1)
    elif classifier == 'xgb':
        clf = xgb.XGBClassifier(booster = 'gbtree', nthread = -1, subsample = 1, n_estimators = 600, colsample_bytree = 1, max_depth = 3, min_child_weight = 1)
    else:
        clf = None
        
    return clf

def evaluate(test_y, pred_y):
    precision = metrics.precision_score(test_y, pred_y, average=None) 
    recall = metrics.recall_score(test_y, pred_y, average=None)
    score = metrics.accuracy_score(test_y, pred_y)
    print('precision=%s, recall=%s, accuracy=%.2f'%(precision,recall, score))

In [35]:
#load data
stagedata = pd.read_csv('stage-2018.csv')
stagedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 805 entries, 0 to 804
Data columns (total 18 columns):
Unnamed: 0            805 non-null int64
target                805 non-null int64
eventid               805 non-null int64
car_number            805 non-null int64
stageid               805 non-null int64
firststage            805 non-null int64
pit_in_caution        805 non-null int64
start_position        805 non-null int64
start_rank            805 non-null int64
start_rank_ratio      805 non-null float64
top_pack              805 non-null int64
bottom_pack           805 non-null int64
average_rank          805 non-null float64
average_rank_all      805 non-null float64
change_in_rank        805 non-null int64
change_in_rank_all    805 non-null float64
rate_of_change        805 non-null int64
rate_of_change_all    805 non-null float64
dtypes: float64(5), int64(13)
memory usage: 113.3 KB


In [ ]:
#events = ['Phoenix','Indy500','Texas','Iowa','Pocono','Gateway']
events = list(stagedata['eventid'])
train = stagedata[stagedata['eventid']!=1].to_numpy()
test  = stagedata[stagedata['eventid']==1].to_numpy()

train_x = train[:,2:]
train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
test_x = test[:,2:]
test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])

In [39]:
### baseline

#1. predict with current rank, rankchg = 0
print('1. predict with current rank, rankchg = 0')
pred_y_simple = np.zeros_like(test_y)
evaluate(test_y, pred_y_simple)

#2. predict with average rankchg (change_in_rank_all):idx = 15
print('2. predict with average rankchg (change_in_rank_all):idx = 15')
change_in_rank_all = test[:,15]
pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in change_in_rank_all])
evaluate(test_y, pred_y_avg)


1. predict with current rank, rankchg = 0
precision=[0.         0.20888889 0.        ], recall=[0. 1. 0.], accuracy=0.21
2. predict with average rankchg (change_in_rank_all):idx = 15
precision=[0.4017094  0.24528302 0.23636364], recall=[0.48958333 0.27659574 0.15853659], accuracy=0.32


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
### test learning models
clf = get_classifier()
clf.fit(train_x, train_y)

pred_y = clf.predict(test_x)
precision = metrics.precision_score(test_y, pred_y, average=None) 
recall = metrics.recall_score(test_y, pred_y, average=None)
score = metrics.accuracy_score(test_y, pred_y)
print('precision=%s, recall=%s, accuracy=%.2f'%(precision,recall, score))

precision=[0.56834532 0.33333333 0.58441558], recall=[0.82291667 0.06382979 0.54878049], accuracy=0.56


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 48.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
